# Info 

This script is mean to perform the lineage reconstruction of the different explant trees using the greedy and neighbor joining methods from cassiopeia.

The hybrid method was completed using seperate scripts due to the length it took to run each hybrid reconstruction step

In [2]:
import cassiopeia as cas
import pandas as pd
import numpy as np
import pickle
import os

In [ ]:
output = '/Genomics/chanlab/blaw/TLS/data/explant/lineage/3_lineage_reconstruction/'
alleleTable_filepath = '/Genomics/chanlab/blaw/TLS/data/explant/lineage/2_add_metadata/merged_allele_table_with_metadata.txt'

barcodes = ['Bar1', 'Bar2', 'Bar3', 'Bar4', 'Bar5', 'Bar6']

# Read in the allele table
allele_table = pd.read_csv(alleleTable_filepath, sep='\t')
allele_table.drop(columns = ['Unnamed: 0'], inplace = True)

# read in the cell state tables
explant_cell_state = pd.read_csv('/Genomics/chanlab/blaw/TLS/metadata/AM-RNA-929_cellBC_cellState.tsv', sep='\t')
outgrowth_1_cell_state = pd.read_csv('/Genomics/chanlab/blaw/TLS/metadata/AM-RNA-930_cellBC_cellState.tsv', sep='\t')
outgrowth_2_cell_state = pd.read_csv('/Genomics/chanlab/blaw/TLS/metadata/AM-RNA-931_cellBC_cellState.tsv', sep='\t')

# Organize and merge the cell state tables with the allele table

In [10]:
explant_cell_state['cellBC'] = ['Tracer_Explant_' + i[:-2] for i in explant_cell_state['cellBC']]

outgrowth_1_cell_state['cellBC'] = [i[:-2] for i in outgrowth_1_cell_state['cellBC']]
outgrowth_2_cell_state['cellBC'] = [i[:-2] for i in outgrowth_2_cell_state['cellBC']]

explant_cell_state.set_index('cellBC', inplace = True)
outgrowth_1_cell_state.set_index('cellBC', inplace = True)
outgrowth_2_cell_state.set_index('cellBC', inplace = True)

allele_table['cell_state'] = ''
cell_states = []

for i in allele_table.index:
    cellBC = allele_table.loc[i, 'cellBC']
    
    if cellBC.startswith('Tracer_Explant'):
        cell_states.append(explant_cell_state.loc[cellBC, 'cell_state'])
    elif cellBC.startswith('Tracer_Outgrowth_1'):
        cell_states.append(outgrowth_1_cell_state.loc[cellBC, 'cell_state'])
    elif cellBC.startswith('Tracer_Outgrowth_2'):
        cell_states.append(outgrowth_2_cell_state.loc[cellBC, 'cell_state'])
        
allele_table['cell_state'] = cell_states

# Seperate edited and unedited cellBC

In [12]:
# Remove any cell that is not edited
edited_cellBC = set()
unedited_cellBC = set()

for cellBC in allele_table['cellBC'].unique():
    bad = True
    for i in allele_table[allele_table['cellBC'] == cellBC]['allele']:
        if i != '[None][None][None]':
            bad = False
    
    if bad:
        unedited_cellBC.add(cellBC)
    else:
        edited_cellBC.add(cellBC)

tableFiltered = allele_table[allele_table['cellBC'].isin(edited_cellBC)]

In [ ]:
tableFiltered.to_csv(output + 'allele_table_filtered.txt', sep = '\t')

In [ ]:
tableFiltered = pd.read_csv(output + 'allele_table_filtered.txt', sep = '\t', index_col = 0)

# Generate lineage and prior tables for cassiopeia

In [ ]:
# Save the lineage table
lineage_table = cas.pp.convert_alleletable_to_lineage_profile(tableFiltered)
lineage_table.to_csv(output + 'Total_Explant_Lineage_Table.txt', sep='\t')

In [ ]:
# Remove any rows with NaN alleles, since they disrupt Cassiopeia
def remove_NaN(table):
    '''
    input:
        table - an allele_table that contains an r1, r2, and r3 cute site column
    output:
        returns a new table with any rows that contain NaN removed
    '''
    table = table[table['r1'].isnull() == False]
    table = table[table['r2'].isnull() == False]
    table = table[table['r3'].isnull() == False]
    
    return table

In [ ]:
### Calculate the total indels using allele_tables from TLS1, TLS2, and TLSCL combined
TLSCL_filepath = '/Genomics/chanlab/blaw/TLS/sandbox/AM-DNA-258/2_add_metadata/allele_table_multiSeq+lenti.txt'
TLS1_filepath = '/Genomics/chanlab/blaw/TLS/sandbox/AM-DNA-097/1_preprocessing/allele_table.txt'
TLS2_filepath = '/Genomics/chanlab/blaw/TLS/sandbox/AM-DNA-098/1_preprocessing/allele_table.txt'
explant_filepath = '/Genomics/chanlab/blaw/TLS/sandbox/explant/2_add_metadata/merged_allele_table_with_metadata.txt'

TLSCL_allele = pd.read_csv(TLSCL_filepath, sep='\t')
TLS1_allele = pd.read_csv(TLS1_filepath, sep='\t')
TLS2_allele = pd.read_csv(TLS2_filepath, sep='\t')
explant_allele = pd.read_csv(explant_filepath, sep = '\t')

# Combine all three datasets
TLSCL_allele = TLSCL_allele[(TLSCL_allele['finalCalls'] != 'Doublet') & (TLSCL_allele['finalCalls'] != 'Negative')]
TLS1_allele['finalCalls'] = 'Bar25'
TLS2_allele['finalCalls'] = 'Bar26'
explant_allele['finalCalls'] = ['explant_' + i for i in explant_allele['finalCalls']]
total_table = pd.concat([TLSCL_allele, TLS1_allele, TLS2_allele, explant_allele])
total_table = remove_NaN(total_table)

# Calculate the priors using each intBC / Multiseq Barcode as a different indel instance
total_priors = cas.pp.compute_empirical_indel_priors(total_table, grouping_variables=['intBC', 'finalCalls'])

# Lineage reconstruction
For each multiseq dataset, I will generate trees for the 120, 144, 120_144 datasets

In [6]:
def greedy_tree_pipeline(table, name, total_priors, output_loc):
    character_matrix, priors, state_2_indel = cas.pp.convert_alleletable_to_character_matrix(table,
                                                                                         allele_rep_thresh = 1,
                                                                                         mutation_priors = total_priors)  

    character_matrix.to_csv(output_loc + 'greedy/' + name + '_character_matrix.txt', sep='\t')
    
    with open(output_loc + 'greedy/' + name + '_priors.pickle', 'wb') as handle:
        pickle.dump(priors, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(output_loc + 'greedy/' + name + '_state_2_indel.pickle', 'wb') as handle:
        pickle.dump(state_2_indel, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix, priors=priors)
    print(cas_tree.n_cell, cas_tree.n_character, '\n')
    
    missing_proportion = (character_matrix == -1).sum(axis=0) / character_matrix.shape[0]
    uncut_proportion = (character_matrix == 0).sum(axis=0) / character_matrix.shape[0]
    n_unique_states = character_matrix.apply(lambda x: len(np.unique(x[(x != 0) & (x != -1)])), axis=0)

    cell_meta = table.groupby('cellBC').agg({"intBC": 'nunique', 'UMI': 'sum', 'cell_state': 'unique', 'Timepoint': 'unique', 'orig.ident':'unique'})
    character_meta = pd.DataFrame([missing_proportion, uncut_proportion, n_unique_states], index = ['missing_prop', 'uncut_prop', 'n_unique_states']).T

    cas_tree.cell_meta = cell_meta
    cas_tree.character_meta = character_meta
    cas_tree.cell_meta.to_csv(output_loc + 'greedy/' + name + '_metadata.txt', sep='\t')
    
    cas_tree.cell_meta['cell_state'] = cas_tree.cell_meta['cell_state'].astype('str')
    
    vanilla_greedy = cas.solver.VanillaGreedySolver()
    vanilla_greedy.solve(cas_tree)
    
    newickFile = open(output_loc + 'greedy/' + name + '_greedy_newick.txt', 'wt')
    newickFile.write(cas_tree.get_newick())
    newickFile.close()
    
    cas_tree.collapse_mutationless_edges(infer_ancestral_characters=True)
    
    noMutationlessEdgesFile = open(output_loc + 'greedy/' + name + '_greedy_newick_noMutationlessEdges.txt', 'wt')
    noMutationlessEdgesFile.write(cas_tree.get_newick())
    noMutationlessEdgesFile.close()

In [9]:
def NJ_tree_pipeline(table, name, total_priors, output_loc):
    character_matrix, priors, state_2_indel = cas.pp.convert_alleletable_to_character_matrix(table,
                                                                                         allele_rep_thresh = 1,
                                                                                         mutation_priors = total_priors)  

    character_matrix.to_csv(output_loc + 'neighbor/' + name + '_character_matrix.txt', sep='\t')

    with open(output_loc + 'neighbor/' + name + '_priors.pickle', 'wb') as handle:
        pickle.dump(priors, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(output_loc + 'neighbor/' + name + '_state_2_indel.pickle', 'wb') as handle:
        pickle.dump(state_2_indel, handle, protocol=pickle.HIGHEST_PROTOCOL)

    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix, priors=priors)
    print(cas_tree.n_cell, cas_tree.n_character, '\n')
    
    missing_proportion = (character_matrix == -1).sum(axis=0) / character_matrix.shape[0]
    uncut_proportion = (character_matrix == 0).sum(axis=0) / character_matrix.shape[0]
    n_unique_states = character_matrix.apply(lambda x: len(np.unique(x[(x != 0) & (x != -1)])), axis=0)

    cell_meta = table.groupby('cellBC').agg({"intBC": 'nunique', 'UMI': 'sum', 'cell_state': 'unique', 'Timepoint': 'unique', 'orig.ident':'unique'})
    character_meta = pd.DataFrame([missing_proportion, uncut_proportion, n_unique_states], index = ['missing_prop', 'uncut_prop', 'n_unique_states']).T

    cas_tree.cell_meta = cell_meta
    cas_tree.character_meta = character_meta
    cas_tree.cell_meta.to_csv(output_loc + 'neighbor/' + name + '_metadata.txt', sep='\t')

    cas_tree.cell_meta['cell_state'] = cas_tree.cell_meta['cell_state'].astype('str')
    
    nj_solver = cas.solver.NeighborJoiningSolver(dissimilarity_function=cas.solver.dissimilarity.weighted_hamming_distance, add_root=True)
    nj_solver.solve(cas_tree, collapse_mutationless_edges=True)

    newickFile = open(output_loc + 'neighbor/' + name + '_neighbor_newick.txt', 'wt')
    newickFile.write(cas_tree.get_newick())
    newickFile.close()

    cas_tree.collapse_mutationless_edges(infer_ancestral_characters=True)

    noMutationlessEdgesFile = open(output_loc + 'neighbor/' + name + '_neighbor_newick_noMutationlessEdges.txt', 'wt')
    noMutationlessEdgesFile.write(cas_tree.get_newick())
    noMutationlessEdgesFile.close()

# Generate greedy trees

In [ ]:
# check if the output file locations exist
for barcode in barcodes:
    for time in [['120h'], ['144h'], ['120h', '144h']]:
        if len(time) > 1:
            timepoint = '{}_{}'.format(time[0][:-1], time[1][:-1])
        else:
            timepoint = '{}'.format(time[0][:-1])
            
        if not os.path.exists('/Genomics/chanlab/blaw/TLS/data/explant/lineage/3_lineage_reconstruction/{}/{}/greedy/'.format(barcode, timepoint)):
            os.mkdir('/Genomics/chanlab/blaw/TLS/data/explant/lineage/3_lineage_reconstruction/{}/{}/greedy/'.format(barcode, timepoint))

In [28]:
# run the greedy algorithm
for barcode in barcodes:
    for time in [['120h'], ['144h'], ['120h', '144h']]:
        temp_tableFiltered = tableFiltered[(tableFiltered['finalCalls'] == barcode) & (tableFiltered['Timepoint'].isin(time))].copy()
        
        temp_tableFiltered = remove_NaN(temp_tableFiltered)

        #temp_tableFiltered = remove_NaN(temp_tableFiltered)
        if len(time) > 1:
            timepoint = '{}_{}'.format(time[0][:-1], time[1][:-1])
        else:
            timepoint = '{}'.format(time[0][:-1])
            
        temp_tableFiltered.to_csv(output + '{}/{}/{}_{}_allele_table.txt'.format(barcode, timepoint, barcode, timepoint), sep = '\t')
        # I will use the best see for TLS1 for now
        if not os.path.exists('/Genomics/chanlab/blaw/TLS/data/explant/lineage/3_lineage_reconstruction/{}/{}/greedy/'.format(barcode, timepoint)):
            os.mkdir('/Genomics/chanlab/blaw/TLS/data/explant/lineage/3_lineage_reconstruction/{}/{}/greedy/'.format(barcode, timepoint))
        
        greedy_tree_pipeline(table = temp_tableFiltered,
                             name = '{}_{}'.format(barcode, timepoint), 
                             total_priors = total_priors,
                             output_loc = '/Genomics/chanlab/blaw/TLS/data/explant/lineage/3_lineage_reconstruction/{}/{}/'.format(barcode, timepoint))

# Generate neighbor joining trees

In [9]:
# check if the output file locations exist
for barcode in barcodes:
    for time in [['120h'], ['144h'], ['120h', '144h']]:
        if len(time) > 1:
            timepoint = '{}_{}'.format(time[0][:-1], time[1][:-1])
        else:
            timepoint = '{}'.format(time[0][:-1])
            
        if not os.path.exists('/Genomics/chanlab/blaw/TLS/data/explant/lineage/3_lineage_reconstruction/{}/{}/neighbor/'.format(barcode, timepoint)):
            os.mkdir('/Genomics/chanlab/blaw/TLS/data/explant/lineage/3_lineage_reconstruction/{}/{}/neighbor/'.format(barcode, timepoint))

In [10]:
for barcode in barcodes:
    for time in [['120h'], ['144h'], ['120h', '144h']]:
        temp_tableFiltered = tableFiltered[(tableFiltered['finalCalls'] == barcode) & (tableFiltered['Timepoint'].isin(time))].copy()
        
        temp_tableFiltered = remove_NaN(temp_tableFiltered)

        #temp_tableFiltered = remove_NaN(temp_tableFiltered)
        if len(time) > 1:
            timepoint = '{}_{}'.format(time[0][:-1], time[1][:-1])
        else:
            timepoint = '{}'.format(time[0][:-1])
            
        #temp_tableFiltered.to_csv(output + '{}/{}/{}_{}_allele_table.txt'.format(barcode, timepoint, barcode, timepoint), sep = '\t')

        NJ_tree_pipeline(table = temp_tableFiltered,
                         name = '{}_{}'.format(barcode, timepoint), 
                         total_priors = total_priors,
                         output_loc = '/Genomics/chanlab/blaw/TLS/sandbox/explant/3_lineage_reconstruction/{}/{}/'.format(barcode, timepoint))

Dropping the following intBCs due to lack of diversity with threshold 1: []


Processing characters:   0%|          | 0/30 [00:00<?, ?it/s]

2080 30 

Dropping the following intBCs due to lack of diversity with threshold 1: []


Processing characters:   0%|          | 0/30 [00:00<?, ?it/s]

4679 30 

Dropping the following intBCs due to lack of diversity with threshold 1: []


...

243 30 

Dropping the following intBCs due to lack of diversity with threshold 1: []


Processing characters:   0%|          | 0/30 [00:00<?, ?it/s]

443 30 

Dropping the following intBCs due to lack of diversity with threshold 1: []


Processing characters:   0%|          | 0/30 [00:00<?, ?it/s]

686 30 

